# Team Challenge Image Analysis - Team 6

This notebook shows an example workflow for the proposed image analysis software, it consists of two parts: training and evaluation. More information about the setup and file structure can be found on the Github page.

## Training

In [ ]:
# Change working directory to the root folder
import os, sys
if os.path.split(os.getcwd())[-1] != 'TeamChallenge_team6':
    %cd ..
    sys.path.append("src")
    
    if os.path.split(os.getcwd())[-1] != 'TeamChallenge_team6':
        raise UserError("Something went wrong in the directory reassignment!")

In [ ]:
# Relevant imports
import sys
if "" not in sys.path : sys.path.append("")

import os
from data_preperation import data_prep
from preprocessing import preprocess_data
from model import define_discriminator, define_generator, define_gan
from training import train
from evaluation import evaluate, get_fnirt_DSC, resp_vec_correlation
from util.tf_session import setup_tf_session

In [ ]:
# Setup the tf session for possible gpu usage
setup_tf_session()

# Preprocess data
print("Step 0: Preprocessing data...\t", end="", flush=True)
preprocess_data("data")
print("Completed!\n")

In [ ]:
# Load data
print("Step 1: Loading and extracting data...\n")

print("Dataset - TRAIN")
dataset_train, train_subjects = data_prep(os.path.join("data", "preprocessed"), True, "train")
print("Dataset - TEST")
dataset_test, test_subjects = data_prep(os.path.join("data", "preprocessed"), True, "test")

image_shape = dataset_train[0].shape[1:]
image_shape = (image_shape[0], image_shape[1], 1)

print("Completed data loading!\n")

In [ ]:
# Define the models
print("Step 2: Defining models")
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)

gan_model = define_gan(g_model, d_model, image_shape)

print("Defining models completed!\n")

In [ ]:
# Train model
print("Step 3: Training")
time = train(d_model, g_model, gan_model, dataset_train, n_epochs=1)
print("Training completed!\n")

## Evaluation

Based on the training we did above, we can evaluate the best performing model. First choose which model (based on step) you want to evaluate (e.g. "0029400"), you can do this by typing the following into your prompt (with specified path to logs): 

tensorboard --logdir "../logs" 

and go to http://localhost:6006/

The step which resulted in the best performing step can be specified in the specific_model parameter below to evaluate the correpsonding model ("last" argument results in evaluating the model from the last step).

In [ ]:
print("Step 4: Evaluation")
eval_SSIMs = evaluate(d_model, g_model, gan_model, dataset_test, time, specific_model="0000003")
eval_DSCs = get_fnirt_DSC("data", test_subjects)
resp_vec_cor = resp_vec_correlation("data", test_subjects, eval_SSIMs)
print("Evaluation completed!\n")

In [ ]:
# TODO:
# correlatie dingen